<a href="https://colab.research.google.com/github/Worachet-Ch/project2022/blob/main/random_nan_and_imputation_with_Height_variable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd #นำเข้า pandas แทนที่ด้วย pd
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import missingno as mno
import missingno as msno
from sklearn import linear_model
%matplotlib inline
import matplotlib
import sklearn 
from sklearn.impute import KNNImputer
from sklearn.metrics import mean_squared_error
import time

In [2]:
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path = '/content/drive/My Drive/project' #เข้า drive
data_file_path = os.path.join(path,'data_notmissing.csv') #ไปที่ไฟล์ confirmed-cases-since-280265.csv
data = pd.read_csv(data_file_path) #อ่านไฟล์ csv

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
data.drop('Unnamed: 0',1,inplace = True)

<ipython-input-3-3830d086682f>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  data.drop('Unnamed: 0',1,inplace = True)


In [4]:
data

,Sex,AgeSYear,telomere length (kb),Weight,Height,BMI,Systolic,Diastolic,HeartRate,SMM,...,003_Creatinine,004_Uric Acid,005_Cholesterol,006_Triglyceride,008_HDL-C,009_LDL,010_AST,012_ALT,013_Alkaline Phos,071_HbA1c
0,2,57.0,6.36,47.4,153.0,20.2,114.0,75.0,69.0,19.3,...,1.0,5.0,176.0,51.0,72.0,94.0,16.0,16.0,62.0,5.1
1,2,24.0,9.71,53.2,157.0,21.6,103.0,69.0,78.0,21.3,...,0.8,5.2,159.0,45.0,52.0,98.0,13.0,17.0,50.0,5.4
2,2,62.0,6.17,54.4,165.0,20.0,95.0,75.0,83.0,21.6,...,0.7,4.1,230.0,94.0,61.0,151.0,18.0,31.0,76.0,5.6
3,2,35.0,7.23,56.3,161.0,21.7,95.0,57.0,72.0,22.5,...,0.6,5.0,198.0,47.0,68.0,120.0,16.0,20.0,35.0,5.1
4,1,46.0,6.52,72.3,170.0,25.0,133.0,77.0,107.0,29.2,...,0.9,6.9,194.0,63.0,60.0,122.0,15.0,19.0,69.0,5.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
148,1,60.0,5.81,72.0,170.0,24.9,131.0,74.0,69.0,30.5,...,0.9,6.9,182.0,179.0,39.0,107.0,36.0,31.0,79.0,6.5
149,1,58.0,5.05,62.0,168.0,22.0,103.0,76.0,72.0,27.8,...,1.0,6.8,191.0,62.0,47.0,132.0,16.0,33.0,42.0,6.8
150,2,42.0,9.48,55.5,165.0,20.4,102.0,52.0,77.0,22.9,...,0.8,4.7,212.0,68.0,63.0,135.0,11.0,16.0,33.0,4.8
151,2,61.0,6.06,61.8,159.0,24.4,117.0,60.0,72.0,23.4,...,0.8,5.1,235.0,66.0,56.0,166.0,15.0,23.0,44.0,5.4


In [ ]:
t_start_t = time.time()

# 10 % random nan

In [6]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'Height10_{i}'] = data.copy() # for loop variable name
  np.random.seed(i) # for loop seed
  n = int((153*10/100)) 
  index_nan = np.random.choice(globals()[f'Height10_{i}']['Height'].size, n, replace=False) #globals()[name dataframe][column name].size
  globals()[f'Height10_{i}']['Height'].ravel()[index_nan] = np.nan
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 0.2069249153137207 วินาที
เวลาที่ใช้ในการประมวลผล 0.003448748588562012 นาที


In [ ]:
Height10_1

## Imputation using Mean

In [8]:
from sklearn.impute import SimpleImputer

In [9]:
t_start = time.time()
mean10_imputer = SimpleImputer(strategy='mean')
for i in range(1,1001):
  globals()[f'imputed_mean10_{i}'] = mean10_imputer.fit_transform(globals()[f'Height10_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MEAN10_{i}'] = mean_squared_error(data, globals()[f'imputed_mean10_{i}']) # for loop mean_squared_error
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 4.7093305587768555 วินาที
เวลาที่ใช้ในการประมวลผล 0.07848884264628092 นาที


In [10]:
t_start = time.time()
mse_mean10_total = 0 
MSE_MEAN10_1001 = 0
for i in range(0,1001):
  mse_mean10_total = mse_mean10_total + globals()[f'MSE_MEAN10_{i+1}']
AMSE_Height10_mean = mse_mean10_total/1000
print(f'AMSE_Height10_mean = {AMSE_Height10_mean}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_Height10_mean = 0.2278241493116809
เวลาที่ใช้ในการประมวลผล 0.003000020980834961 วินาที
เวลาที่ใช้ในการประมวลผล 5.000034968058268e-05 นาที


## Imputation using the miss forest

In [11]:
import sys
import sklearn.neighbors._base
sys.modules['sklearn.neighbors.base'] = sklearn.neighbors._base
!pip install missingpy
import missingpy
from missingpy import MissForest
from sklearn.metrics import mean_squared_error

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 49 kB 3.3 MB/s 


In [ ]:
t_start = time.time()
MF10_imputer = MissForest()
for i in range(1,1001):
  globals()[f'imputed_mf10_{i}'] = MF10_imputer.fit_transform(globals()[f'Height10_{i}'].copy()) #for loop miss forest impputation 
  globals()[f'MSE_MF10_{i}'] = mean_squared_error(data, globals()[f'imputed_mf10_{i}']) # for loop mean_squared_error
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

In [13]:
t_start = time.time()
mse_mf10_total = 0 
MSE_MF10_1001 = 0
for i in range(0,1001):
  mse_mf10_total = mse_mf10_total + globals()[f'MSE_MF10_{i+1}']
AMSE_Height10_mf = mse_mf10_total/1000
print(f'AMSE_Height10_mf = {AMSE_Height10_mf}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_Height10_mf = 0.044141675163398744
เวลาที่ใช้ในการประมวลผล 0.008220672607421875 วินาที
เวลาที่ใช้ในการประมวลผล 0.0001370112101236979 นาที


## Imputation using the KNN

In [14]:
from sklearn import neighbors
from sklearn.impute import KNNImputer

In [15]:
t_start = time.time()
KNN10_imputer = KNNImputer(n_neighbors = 15)
for i in range(1,1001):
  globals()[f'imputed_KNN10_{i}'] = KNN10_imputer.fit_transform(globals()[f'Height10_{i}'].copy())
  globals()[f'MSE_KNN10_{i}'] = mean_squared_error(data, globals()[f'imputed_KNN10_{i}'])
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 7.515119314193726 วินาที
เวลาที่ใช้ในการประมวลผล 0.12525198856989542 นาที


In [16]:
t_start = time.time()
mse_knn10_total = 0 
MSE_KNN10_1001 = 0
for i in range(0,1001):
  mse_knn10_total = mse_knn10_total + globals()[f'MSE_KNN10_{i+1}']
AMSE_Height10_knn = mse_knn10_total/1000
print(f'AMSE_Height10_knn = {AMSE_Height10_knn}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_Height10_knn = 0.20476123080233452
เวลาที่ใช้ในการประมวลผล 0.026914119720458984 วินาที
เวลาที่ใช้ในการประมวลผล 0.00044856866200764975 นาที


## Imputation using the MICE

In [17]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression
from sklearn import svm

### LinearRegression model

In [ ]:
t_start = time.time()
lr = LinearRegression()
for i in range(1,1001):
  imp = IterativeImputer(estimator= lr , max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_lr10_{i}'] = imp.fit_transform(globals()[f'Height10_{i}'].copy())
  globals()[f'MSE_lr10_{i}'] = mean_squared_error(data, globals()[f'imputed_lr10_{i}'])
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

In [19]:
t_start = time.time()
mse_lr10_total = 0 
MSE_lr10_1001 = 0
for i in range(0,1001):
  mse_lr10_total = mse_lr10_total + globals()[f'MSE_lr10_{i+1}']
AMSE_Height10_lr = mse_lr10_total/1000
print(f'AMSE_Height10_lr = {AMSE_Height10_lr}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_Height10_lr = 0.010499577775380776
เวลาที่ใช้ในการประมวลผล 0.007462739944458008 วินาที
เวลาที่ใช้ในการประมวลผล 0.00012437899907430012 นาที


### Support Vector Machine model

In [ ]:
t_start = time.time()
regr = svm.SVR()
for i in range(1,1001):
  svm_svr = IterativeImputer(estimator= regr, max_iter= 10, verbose=2, imputation_order='roman',initial_strategy = 'mean',random_state= i)
  globals()[f'imputed_svm10_{i}'] = svm_svr.fit_transform(globals()[f'Height10_{i}'].copy())
  globals()[f'MSE_svm10_{i}'] = mean_squared_error(data, globals()[f'imputed_svm10_{i}'])
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

In [21]:
t_start = time.time()
mse_svm10_total = 0 
MSE_svm10_1001 = 0
for i in range(0,1001):
  mse_svm10_total = mse_svm10_total + globals()[f'MSE_svm10_{i+1}']
AMSE_Height10_svm = mse_svm10_total/1000
print(f'AMSE_Height10_svm = {AMSE_Height10_svm}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_Height10_svm = 0.21545184062851444
เวลาที่ใช้ในการประมวลผล 0.003963947296142578 วินาที
เวลาที่ใช้ในการประมวลผล 6.606578826904297e-05 นาที


## Imputation using the EM

### impy.em

In [22]:
!pip install impyute
import impyute as impy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [23]:
t_start = time.time()
for i in range(1,1001):
  globals()[f'imputed_impy10_{i}'] = impy.em(globals()[f'Height10_{i}'].values.copy())
  globals()[f'MSE_impy10_{i}'] = mean_squared_error(data, globals()[f'imputed_impy10_{i}'])
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

เวลาที่ใช้ในการประมวลผล 9.871711015701294 วินาที
เวลาที่ใช้ในการประมวลผล 0.1645285169283549 นาที


In [24]:
t_start = time.time()
mse_impy10_total = 0 
MSE_impy10_1001 = 0
for i in range(0,1001):
  mse_impy10_total = mse_impy10_total + globals()[f'MSE_impy10_{i+1}']
AMSE_Height10_impy = mse_impy10_total/1000
print(f'AMSE_Height10_impy = {AMSE_Height10_impy}')
t_end = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end- t_start} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end- t_start)/60} นาที')

AMSE_Height10_impy = 0.42705194608469244
เวลาที่ใช้ในการประมวลผล 0.012519598007202148 วินาที
เวลาที่ใช้ในการประมวลผล 0.00020865996678670246 นาที


# เวลาทั้งหมดในการรัน

In [ ]:
t_end_t = time.time()
print(f'เวลาที่ใช้ในการประมวลผล {t_end_t - t_start_t} วินาที')
print(f'เวลาที่ใช้ในการประมวลผล {(t_end_t- t_start_t)/60} นาที')